In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
df = pd.read_csv("../input/flavors_of_cacao.csv")
df.head()

Convert Cocoa Percent to float for charting.

In [ ]:
df['Cocoa\nPercent'] = df['Cocoa\nPercent'].apply(lambda x : float(x[:-1]))

Based on my analysis I have found some issues with the dataset and let's correct them here.

In [ ]:
columnNames = df.columns.values
countryCol = columnNames[5]
originCol = columnNames[8]
df.replace(to_replace = {countryCol : {'Amsterdam' : 'Netherlands',
                                        'Eucador' : 'Ecuador'},
                         originCol : {'Domincan Republic' : 'Dominican Republic'}}, inplace = True)

In [ ]:
import seaborn as sns
fig, ax = sns.mpl.pyplot.subplots(figsize=(15,6))
sns.distplot(df['Rating'])

The distribution of ratings in the dataset looks healthy and not skewed in any particular direction. Cocoa Percent could be a good indicator of Rating. Let's check how these two are related on a swarmplot.

In [ ]:
cocoaPct = columnNames[4]
fig, ax = sns.mpl.pyplot.subplots(figsize=(15,6)) #fig, ax = sns.mpl.pyplot.subplots()
sns.swarmplot(ax = ax, y = cocoaPct, x = 'Rating', data = df, orient = 'v')

Observations:
1. Highest ratings are obtained by chocolates in a narrow 60 - 80 Cocoa Percent range
2. 90% or higher Cocoa Percent does not get very good ratings. Could this be because they are just too bitter ?
3. There are all kinds of %ages in the 2.5 - 3.5 Rating range
4. A perfect rating of 5 is only obtained by Chocolates with 70% Cocoa

In the next step let's analyze how are the ratings affected by Company location. Do chocolates from European countries really get higher ratings ?

In [ ]:
dfGrouped = df.groupby(countryCol)
nReviews = dfGrouped.size()
fig, ax = sns.mpl.pyplot.subplots(figsize=(15,15))
sns.countplot(ax = ax, y = countryCol, data = df, orient = 'h', order = nReviews.sort_values(ascending = False).index.values)

Looks like a majority of the chocolates are manufactured by US and French companies. There are many countries with very few Chocolates in the database and most of them seem consistent with common knowledge. Some European countries do not have a good number (Czech Republic, Sweden, Finland) but that might be possible. 

The dataset seems to have a disproportionate number of US manufacturers, I was expecting to see more from European nations. In the next step let's loook at the average ratings based on company location with a minimum of 10 reviews from the country.  

In [ ]:
meanRating = dfGrouped['Rating'].mean()
meanRating = meanRating[nReviews > 10].sort_values(ascending = False)

fig, ax = sns.mpl.pyplot.subplots(figsize=(15,10))
sns.boxplot(ax = ax, x =  'Rating', y = columnNames[5], data = df, order = meanRating.index.values, color = 'yellow')


Surprisingly, 
1. Vietnam manufacturers seem to have the highest average rating.
2. Switzerland is pretty good as expected
3. Belgian manufacturers don't have a very good average rating.
4. Brazil and Australia are kind of unexpected, Australia very unexpected because they are not close to any Cocoa producing places. 

Next let's analyze based on bean origin.


In [ ]:
originCol = columnNames[8]
nReviewsByOrigin = df.groupby(originCol).size().sort_values(ascending = False)
fig, ax = sns.mpl.pyplot.subplots(figsize=(15,25))
sns.countplot(ax = ax, y = originCol, data = df, orient = 'h', order = nReviewsByOrigin.index.values)

South American countries dominate the list with some African countries as well. Notice the significant Vietnam numbers here. Also, a lot of bean origins seem to be a mix of countries but since the numbers are so low we can ignore the mixed ones. Let's look at the average rating by bean origin. 

In [ ]:
meanByOrigin = df.groupby(originCol)['Rating'].mean()[nReviewsByOrigin > 10].sort_values(ascending = False)
fig, ax = sns.mpl.pyplot.subplots(figsize=(15,10))
sns.boxplot(ax = ax, x =  'Rating', y = originCol, data = df, order = meanByOrigin.index.values, color = 'yellow')

Honduras and Guatemala are the top 2 but Vietnam beans are also ranked pretty high. The higest rating is obtained by a Venezuela bean and one Unknown origin bean.